In [1]:
import random
from scoop import futures
from deap import base, creator, tools, cma

In [2]:
# 従業員を表すクラス
class Employee(object):
  def __init__(self, no, name, age, manager, wills):
    self.no = no
    self.name = name
    self.age = age
    self.manager = manager

    # willは曜日_時間帯。1は朝、2は昼、3は夜。
    # 例）mon_1は月曜日の朝
    self.wills = wills

  def is_applicated(self, box_name):
    return (box_name in self.wills)

# シフトを表すクラス
# 3(朝昼晩) * 7日 * 10人 = 210次元のタプルで構成
class Shift(object):
  # コマの定義
  SHIFT_BOXES = [
    'mon_1', 'mon_2', 'mon_3',
    'tue_1', 'tue_2', 'tue_3',
    'wed_1', 'wed_2', 'wed_3',
    'thu_1', 'thu_2', 'thu_3',
    'fri_1', 'fri_2', 'fri_3',
    'sat_1', 'sat_2', 'sat_3',
    'sun_1', 'sun_2', 'sun_3']

  # 各コマの想定人数
  NEED_PEOPLE = [
    2,3,3,
    2,3,3,
    2,3,3,
    1,2,2,
    2,3,3,
    2,4,4,
    2,4,4]

  def __init__(self, list):
    if list == None:
      self.make_sample()
    else:
      self.list = list
    self.employees = []

  # ランダムなデータを生成
  def make_sample(self):
    sample_list = []
    for num in range(210):
      sample_list.append(random.randint(0, 1))
    self.list = tuple(sample_list)

  # タプルを1ユーザ単位に分割
  def slice(self):
    sliced = []
    start = 0
    for num in range(10):
      sliced.append(self.list[start:(start + 21)])
      start = start + 21
    return tuple(sliced)

  # ユーザ別にアサインコマ名を出力
  def print_inspect(self):
    user_no = 0
    for line in self.slice():
      print("ユーザ%d" % user_no)
      print(line)
      user_no = user_no + 1

      index = 0
      for e in line:
        if e == 1:
          print(self.SHIFT_BOXES[index])
        index = index + 1

  # CSV形式でアサイン結果の出力
  def print_csv(self):
    for line in self.slice():
      print(','.join(map(str, line)))

  # TSV形式でアサイン結果の出力
  def print_tsv(self):
    for line in self.slice():
      print("\t".join(map(str, line)))

  # ユーザ番号を指定してコマ名を取得
  def get_boxes_by_user(self, user_no):
    line = self.slice()[user_no]
    return self.line_to_box(line)

  # 1ユーザ分のタプルからコマ名を取得
  def line_to_box(self, line):
    result = []
    index = 0
    for e in line:
      if e == 1:
        result.append(self.SHIFT_BOXES[index])
      index = index + 1
    return result    

  # コマ番号を指定してアサインされているユーザ番号リストを取得
  def get_user_nos_by_box_index(self, box_index):
    user_nos = []
    index = 0
    for line in self.slice():
      if line[box_index] == 1:
        user_nos.append(index)
      index += 1
    return user_nos

  # コマ名を指定してアサインされているユーザ番号リストを取得
  def get_user_nos_by_box_name(self, box_name):
    box_index = self.SHIFT_BOXES.index(box_name)
    return self.get_user_nos_by_box_index(box_index)

  # 想定人数と実際の人数の差分を取得
  def abs_people_between_need_and_actual(self):
    result = []
    index = 0
    for need in self.NEED_PEOPLE:
      actual = len(self.get_user_nos_by_box_index(index))
      result.append(abs(need - actual))
      index += 1
    return result

  # 応募していないコマにアサインされている件数を取得
  def not_applicated_assign(self):
    count = 0
    for box_name in self.SHIFT_BOXES:
      user_nos = self.get_user_nos_by_box_name(box_name)
      for user_no in user_nos:
        e = self.employees[user_no]
        if not e.is_applicated(box_name):
          count += 1
    return count

  # アサインが応募コマ数の50%に満たないユーザを取得
  def few_work_user(self):
    result = []
    for user_no in range(10):
      e = self.employees[user_no]
      ratio = float(len(self.get_boxes_by_user(user_no))) / float(len(e.wills))
      if ratio < 0.5:
        result.append(e)
    return result

  # 管理者が1人もいないコマ
  def no_manager_box(self):
    result = []
    for box_name in self.SHIFT_BOXES:
      manager_included = False
      user_nos = self.get_user_nos_by_box_name(box_name)
      for user_no in user_nos:
        e = self.employees[user_no]
        if e.manager:
          manager_included = True
      if not manager_included:
        result.append(box_name)
    return result

  # 1日1人3コマの日を返却
  def three_box_per_day(self):
    result = []
    for user_no in range(10):
      boxes = self.get_boxes_by_user(user_no)
      wdays = []
      for box in boxes:
        wdays.append(box.split('_')[0])
      wday_names = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
      for wday_name in wday_names:
        if wdays.count(wday_name) == 3:
          result.append(wday_name)
    return result

# 従業員定義

# 朝だけ
e0 = Employee(0, "山田", 40, False, ['mon_1', 'tue_1', 'wed_1', 'thu_1', 'fri_1', 'sat_1', 'sun_1'])

# 月・水・金
e1 = Employee(1, "鈴木", 21, False, ['mon_1', 'mon_2', 'mon_3', 'wed_1', 'wed_2', 'wed_3','fri_1', 'fri_2', 'fri_3'])

# 週末だけ
e2 = Employee(2, "佐藤", 18, False, ['sat_1', 'sat_2', 'sat_3', 'sun_1', 'sun_2', 'sun_3'])

# どこでもOK
e3 = Employee(3, "田中", 35, True, ['mon_1', 'mon_2', 'mon_3',
                                     'tue_1', 'tue_2', 'tue_3',
                                     'wed_1', 'wed_2', 'wed_3',
                                     'thu_1', 'thu_2', 'thu_3',
                                     'fri_1', 'fri_2', 'fri_3',
                                     'sat_1', 'sat_2', 'sat_3',
                                     'sun_1', 'sun_2', 'sun_3'
                                    ])

# 夜だけ
e4 = Employee(4, "山口", 19, False, ['mon_3', 'tue_3', 'wed_3', 'thu_3', 'fri_3', 'sat_3', 'sun_3'])

# 平日のみ
e5 = Employee(5, "加藤", 43, True, ['mon_1', 'mon_2', 'mon_3',
                                     'tue_1', 'tue_2', 'tue_3',
                                     'wed_1', 'wed_2', 'wed_3',
                                     'thu_1', 'thu_2', 'thu_3',
                                     'fri_1', 'fri_2', 'fri_3'
                                    ])

# 金土日
e6 = Employee(6, "川口", 25, False, ['fri_1', 'fri_2', 'fri_3',
                                     'sat_1', 'sat_2', 'sat_3',
                                     'sun_1', 'sun_2', 'sun_3'
                                    ])

# 昼のみ
e7 = Employee(7, "野口", 22, False, ['mon_2', 'tue_2', 'wed_2', 'thu_2', 'fri_2', 'sat_2', 'sun_2'])

# 夜のみ
e8 = Employee(8, "棚橋", 18, False, ['mon_3', 'tue_3', 'wed_3', 'thu_3', 'fri_3', 'sat_3', 'sun_3'])

# 木金土日
e9 = Employee(9, "小山", 30, True, ['thu_1', 'thu_2', 'thu_3',
                                     'fri_1', 'fri_2', 'fri_3',
                                     'sat_1', 'sat_2', 'sat_3',
                                     'sun_1', 'sun_2', 'sun_3'
                                    ])

employees = [e0, e1, e2, e3, e4, e5, e6, e7, e8, e9]

creator.create("FitnessPeopleCount", base.Fitness, weights=(-10.0, -100.0, -1.0, -100.0, -10.0))
creator.create("Individual", list, fitness=creator.FitnessPeopleCount)

toolbox = base.Toolbox()

toolbox.register("map", futures.map)

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 210)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalShift(individual):
  s = Shift(individual)
  s.employees = employees

  # 想定人数とアサイン人数の差
  people_count_sub_sum = sum(s.abs_people_between_need_and_actual()) / 210.0
  # 応募していない時間帯へのアサイン数
  not_applicated_count = s.not_applicated_assign() / 210.0
  # アサイン数が応募数の半分以下の従業員数
  few_work_user = len(s.few_work_user()) / 10.0
  # 管理者が１人もいないコマ数
  no_manager_box = len(s.no_manager_box()) / 21.0
  # 朝・昼・夜の全部にアサインされている
  three_box_per_day = len(s.three_box_per_day()) / 70.0
  return (not_applicated_count, people_count_sub_sum, few_work_user, no_manager_box, three_box_per_day)

toolbox.register("evaluate", evalShift)
# 交叉関数を定義(二点交叉)
toolbox.register("mate", tools.cxTwoPoint)

# 変異関数を定義(ビット反転、変異隔離が5%ということ?)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# 選択関数を定義(トーナメント選択、tournsizeはトーナメントの数？)
toolbox.register("select", tools.selTournament, tournsize=3)

if __name__ == '__main__':
    # 初期集団を生成する
    pop = toolbox.population(n=300)
    CXPB, MUTPB, NGEN = 0.6, 0.5, 500 # 交差確率、突然変異確率、進化計算のループ回数

    print("進化開始")

    # 初期集団の個体を評価する
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):  # zipは複数変数の同時ループ
        # 適合性をセットする
        ind.fitness.values = fit

    print(("  %i の個体を評価" % len(pop)))

     # 進化計算開始
    for g in range(NGEN):
        print(("-- %i 世代 --" % g))

        # 選択
        # 次世代の個体群を選択
        offspring = toolbox.select(pop, len(pop))
        # 個体群のクローンを生成
        offspring = list(map(toolbox.clone, offspring))

        # 選択した個体群に交差と突然変異を適応する

        # 交叉
        # 偶数番目と奇数番目の個体を取り出して交差
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                # 交叉された個体の適合度を削除
                del child1.fitness.values
                del child2.fitness.values

        # 変異
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # 適合度が計算されていない個体を集めて適合度を計算
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print(("  %i の個体を評価" % len(invalid_ind)))

        # 次世代群をoffspringにする
        pop[:] = offspring

        # すべての個体の適合度を配列
        index = 1
        for v in ind.fitness.values:
          fits = [v for ind in pop]

          length = len(pop)
          mean = sum(fits) / length
          sum2 = sum(x*x for x in fits)
          std = abs(sum2 / length - mean**2)**0.5

          print(("* パラメータ%d") % index)
          print(("  Min %s" % min(fits)))
          print(("  Max %s" % max(fits)))
          print(("  Avg %s" % mean))
          print(("  Std %s" % std))
          index += 1

    print("-- 進化終了 --")

    best_ind = tools.selBest(pop, 1)[0]
    print(("最も優れていた個体: %s, %s" % (best_ind, best_ind.fitness.values)))
    s = Shift(best_ind)
    s.print_csv()
    s.print_tsv()

進化開始
  300 の個体を評価
-- 0 世代 --
  243 の個体を評価
* パラメータ1
  Min 0.24761904761904763
  Max 0.24761904761904763
  Avg 0.24761904761904782
  Std 9.497663962827594e-09
* パラメータ2
  Min 0.2523809523809524
  Max 0.2523809523809524
  Avg 0.2523809523809521
  Std 1.1780402288468106e-08
* パラメータ3
  Min 0.1
  Max 0.1
  Avg 0.10000000000000052
  Std 1.3104870244367317e-08
* パラメータ4
  Min 0.19047619047619047
  Max 0.19047619047619047
  Avg 0.1904761904761899
  Std 1.7865864646854695e-08
* パラメータ5
  Min 0.08571428571428572
  Max 0.08571428571428572
  Avg 0.08571428571428547
  Std 9.125060374972142e-09
-- 1 世代 --
  257 の個体を評価
* パラメータ1
  Min 0.21428571428571427
  Max 0.21428571428571427
  Avg 0.2142857142857149
  Std 9.125060374972142e-09
* パラメータ2
  Min 0.2
  Max 0.2
  Avg 0.20000000000000104
  Std 2.6209740488734633e-08
* パラメータ3
  Min 0.1
  Max 0.1
  Avg 0.10000000000000052
  Std 1.3104870244367317e-08
* パラメータ4
  Min 0.19047619047619047
  Max 0.19047619047619047
  Avg 0.1904761904761899
  Std 1.7865864646854695

  240 の個体を評価
* パラメータ1
  Min 0.10476190476190476
  Max 0.10476190476190476
  Avg 0.10476190476190445
  Std 7.33324128576723e-09
* パラメータ2
  Min 0.15714285714285714
  Max 0.15714285714285714
  Avg 0.15714285714285786
  Std 1.6130980174698657e-08
* パラメータ3
  Min 0.2
  Max 0.2
  Avg 0.20000000000000104
  Std 2.6209740488734633e-08
* パラメータ4
  Min 0.14285714285714285
  Max 0.14285714285714285
  Avg 0.14285714285714338
  Std 1.666000468656264e-08
* パラメータ5
  Min 0.05714285714285714
  Max 0.05714285714285714
  Avg 0.057142857142856975
  Std 5.963374150388733e-09
-- 17 世代 --
  235 の個体を評価
* パラメータ1
  Min 0.12380952380952381
  Max 0.12380952380952381
  Avg 0.12380952380952391
  Std 4.748831981413797e-09
* パラメータ2
  Min 0.1761904761904762
  Max 0.1761904761904762
  Avg 0.17619047619047692
  Std 1.3560261371136677e-08
* パラメータ3
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0
* パラメータ4
  Min 0.09523809523809523
  Max 0.09523809523809523
  Avg 0.09523809523809496
  Std 8.932932323427348e-09
* パラメータ5
  Min 0.0714285

  251 の個体を評価
* パラメータ1
  Min 0.04285714285714286
  Max 0.04285714285714286
  Avg 0.04285714285714273
  Std 4.562530187486071e-09
* パラメータ2
  Min 0.12380952380952381
  Max 0.12380952380952381
  Avg 0.12380952380952391
  Std 4.748831981413797e-09
* パラメータ3
  Min 0.4
  Max 0.4
  Avg 0.4000000000000021
  Std 5.2419480977469267e-08
* パラメータ4
  Min 0.19047619047619047
  Max 0.19047619047619047
  Avg 0.1904761904761899
  Std 1.7865864646854695e-08
* パラメータ5
  Min 0.04285714285714286
  Max 0.04285714285714286
  Avg 0.04285714285714273
  Std 4.562530187486071e-09
-- 32 世代 --
  236 の個体を評価
* パラメータ1
  Min 0.047619047619047616
  Max 0.047619047619047616
  Avg 0.04761904761904748
  Std 4.466466161713674e-09
* パラメータ2
  Min 0.11428571428571428
  Max 0.11428571428571428
  Avg 0.11428571428571395
  Std 1.1926748300777466e-08
* パラメータ3
  Min 0.4
  Max 0.4
  Avg 0.4000000000000021
  Std 5.2419480977469267e-08
* パラメータ4
  Min 0.19047619047619047
  Max 0.19047619047619047
  Avg 0.1904761904761899
  Std 1.786586464

  251 の個体を評価
* パラメータ1
  Min 0.0380952380952381
  Max 0.0380952380952381
  Avg 0.03809523809523799
  Std 1.3170890159654386e-09
* パラメータ2
  Min 0.12857142857142856
  Max 0.12857142857142856
  Avg 0.1285714285714286
  Std 8.33000234328132e-09
* パラメータ3
  Min 0.3
  Max 0.3
  Avg 0.2999999999999983
  Std 2.9802322387695312e-08
* パラメータ4
  Min 0.14285714285714285
  Max 0.14285714285714285
  Avg 0.14285714285714338
  Std 1.666000468656264e-08
* パラメータ5
  Min 0.07142857142857142
  Max 0.07142857142857142
  Avg 0.07142857142857169
  Std 8.33000234328132e-09
-- 47 世代 --
  237 の個体を評価
* パラメータ1
  Min 0.004761904761904762
  Max 0.004761904761904762
  Avg 0.0047619047619047485
  Std 1.6463612699567982e-10
* パラメータ2
  Min 0.10952380952380952
  Max 0.10952380952380952
  Avg 0.10952380952381001
  Std 1.0030651105818346e-08
* パラメータ3
  Min 0.4
  Max 0.4
  Avg 0.4000000000000021
  Std 5.2419480977469267e-08
* パラメータ4
  Min 0.09523809523809523
  Max 0.09523809523809523
  Avg 0.09523809523809496
  Std 8.932932323

  229 の個体を評価
* パラメータ1
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0
* パラメータ2
  Min 0.09047619047619047
  Max 0.09047619047619047
  Avg 0.09047619047619093
  Std 4.16500117164066e-09
* パラメータ3
  Min 0.5
  Max 0.5
  Avg 0.5
  Std 0.0
* パラメータ4
  Min 0.23809523809523805
  Max 0.23809523809523805
  Avg 0.23809523809523928
  Std 2.837096566656198e-08
* パラメータ5
  Min 0.05714285714285714
  Max 0.05714285714285714
  Avg 0.057142857142856975
  Std 5.963374150388733e-09
-- 62 世代 --
  235 の個体を評価
* パラメータ1
  Min 0.009523809523809525
  Max 0.009523809523809525
  Avg 0.009523809523809497
  Std 3.2927225399135965e-10
* パラメータ2
  Min 0.10952380952380952
  Max 0.10952380952380952
  Avg 0.10952380952381001
  Std 1.0030651105818346e-08
* パラメータ3
  Min 0.6
  Max 0.6
  Avg 0.5999999999999966
  Std 5.960464477539063e-08
* パラメータ4
  Min 0.23809523809523805
  Max 0.23809523809523805
  Avg 0.23809523809523928
  Std 2.837096566656198e-08
* パラメータ5
  Min 0.07142857142857142
  Max 0.07142857142857142
  Avg 0.07142857142857169


...

  247 の個体を評価
* パラメータ1
  Min 0.023809523809523808
  Max 0.023809523809523808
  Avg 0.02380952380952374
  Std 2.233233080856837e-09
* パラメータ2
  Min 0.0380952380952381
  Max 0.0380952380952381
  Avg 0.03809523809523799
  Std 1.3170890159654386e-09
* パラメータ3
  Min 0.3
  Max 0.3
  Avg 0.2999999999999983
  Std 2.9802322387695312e-08
* パラメータ4
  Min 0.047619047619047616
  Max 0.047619047619047616
  Avg 0.04761904761904748
  Std 4.466466161713674e-09
* パラメータ5
  Min 0.014285714285714285
  Max 0.014285714285714285
  Avg 0.014285714285714244
  Std 1.4908435375971833e-09
-- 428 世代 --
  247 の個体を評価
* パラメータ1
  Min 0.014285714285714285
  Max 0.014285714285714285
  Avg 0.014285714285714244
  Std 1.4908435375971833e-09
* パラメータ2
  Min 0.02857142857142857
  Max 0.02857142857142857
  Avg 0.028571428571428487
  Std 2.9816870751943665e-09
* パラメータ3
  Min 0.3
  Max 0.3
  Avg 0.2999999999999983
  Std 2.9802322387695312e-08
* パラメータ4
  Min 0.047619047619047616
  Max 0.047619047619047616
  Avg 0.04761904761904748
  S

  240 の個体を評価
* パラメータ1
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0
* パラメータ2
  Min 0.004761904761904762
  Max 0.004761904761904762
  Avg 0.0047619047619047485
  Std 1.6463612699567982e-10
* パラメータ3
  Min 0.3
  Max 0.3
  Avg 0.2999999999999983
  Std 2.9802322387695312e-08
* パラメータ4
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0
* パラメータ5
  Min 0.014285714285714285
  Max 0.014285714285714285
  Avg 0.014285714285714244
  Std 1.4908435375971833e-09
-- 446 世代 --
  243 の個体を評価
* パラメータ1
  Min 0.01904761904761905
  Max 0.01904761904761905
  Avg 0.019047619047618994
  Std 6.585445079827193e-10
* パラメータ2
  Min 0.03333333333333333
  Max 0.03333333333333333
  Avg 0.03333333333333325
  Std 2.9816870751943665e-09
* パラメータ3
  Min 0.2
  Max 0.2
  Avg 0.20000000000000104
  Std 2.6209740488734633e-08
* パラメータ4
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0
* パラメータ5
  Min 0.04285714285714286
  Max 0.04285714285714286
  Avg 0.04285714285714273
  Std 4.562530187486071e-09
-- 447 世代 --
  235 の個体を評価
* パラメータ1
  Min 0.0
  Max 0.0
  Avg 

  252 の個体を評価
* パラメータ1
  Min 0.03333333333333333
  Max 0.03333333333333333
  Avg 0.03333333333333325
  Std 2.9816870751943665e-09
* パラメータ2
  Min 0.04285714285714286
  Max 0.04285714285714286
  Avg 0.04285714285714273
  Std 4.562530187486071e-09
* パラメータ3
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0
* パラメータ4
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0
* パラメータ5
  Min 0.02857142857142857
  Max 0.02857142857142857
  Avg 0.028571428571428487
  Std 2.9816870751943665e-09
-- 464 世代 --
  228 の個体を評価
* パラメータ1
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0
* パラメータ2
  Min 0.014285714285714285
  Max 0.014285714285714285
  Avg 0.014285714285714244
  Std 1.4908435375971833e-09
* パラメータ3
  Min 0.4
  Max 0.4
  Avg 0.4000000000000021
  Std 5.2419480977469267e-08
* パラメータ4
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0
* パラメータ5
  Min 0.04285714285714286
  Max 0.04285714285714286
  Avg 0.04285714285714273
  Std 4.562530187486071e-09
-- 465 世代 --
  244 の個体を評価
* パラメータ1
  Min 0.01904761904761905
  Max 0.01904761904761905
  Avg 0.01904

  241 の個体を評価
* パラメータ1
  Min 0.03333333333333333
  Max 0.03333333333333333
  Avg 0.03333333333333325
  Std 2.9816870751943665e-09
* パラメータ2
  Min 0.05238095238095238
  Max 0.05238095238095238
  Avg 0.05238095238095222
  Std 3.666620642883615e-09
* パラメータ3
  Min 0.1
  Max 0.1
  Avg 0.10000000000000052
  Std 1.3104870244367317e-08
* パラメータ4
  Min 0.047619047619047616
  Max 0.047619047619047616
  Avg 0.04761904761904748
  Std 4.466466161713674e-09
* パラメータ5
  Min 0.014285714285714285
  Max 0.014285714285714285
  Avg 0.014285714285714244
  Std 1.4908435375971833e-09
-- 482 世代 --
  243 の個体を評価
* パラメータ1
  Min 0.047619047619047616
  Max 0.047619047619047616
  Avg 0.04761904761904748
  Std 4.466466161713674e-09
* パラメータ2
  Min 0.06666666666666667
  Max 0.06666666666666667
  Avg 0.0666666666666665
  Std 5.963374150388733e-09
* パラメータ3
  Min 0.2
  Max 0.2
  Avg 0.20000000000000104
  Std 2.6209740488734633e-08
* パラメータ4
  Min 0.047619047619047616
  Max 0.047619047619047616
  Avg 0.04761904761904748
  Std 

  255 の個体を評価
* パラメータ1
  Min 0.023809523809523808
  Max 0.023809523809523808
  Avg 0.02380952380952374
  Std 2.233233080856837e-09
* パラメータ2
  Min 0.05238095238095238
  Max 0.05238095238095238
  Avg 0.05238095238095222
  Std 3.666620642883615e-09
* パラメータ3
  Min 0.3
  Max 0.3
  Avg 0.2999999999999983
  Std 2.9802322387695312e-08
* パラメータ4
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0
* パラメータ5
  Min 0.014285714285714285
  Max 0.014285714285714285
  Avg 0.014285714285714244
  Std 1.4908435375971833e-09
-- 進化終了 --
最も優れていた個体: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0